<a href="https://colab.research.google.com/github/Rachna0604/Object-Lane_Detection/blob/main/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading Packages

In [ ]:
!pip install numpy
!pip install torch
! pip install scipy
! pip install imageio==2.4.1
! pip install imageio-ffmpeg
!pip install pyngrok
!apt-get install -qq xattr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 4.8 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=316c6f522baa9646758f081fa999f2a5b4244f3a3561de8964e741fe7f95988a
  Stored in directory: /root/.cache/pip/wheels/be/7b/04/4d8d56f1d503e5c404f0de6018c0cfa592c71588a39b49e002
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [ ]:
# accessing g drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# cloning repo
!git clone https://github.com/ultralytics/yolov5.git
!pip install -U pycocotools
!pip install -qr yolov5/requirements.txt  # install dependencies
!cp yolov5/requirements.txt ./

Cloning into 'yolov5'...
remote: Enumerating objects: 14302, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 14302 (delta 26), reused 30 (delta 9), pack-reused 14242
Receiving objects: 100% (14302/14302), 13.65 MiB | 29.92 MiB/s, done.
Resolving deltas: 100% (9833/9833), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 4.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 1.6 MB 48.1 MB/s 


## Importing Modules

In [ ]:
import os, time, random
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
import cv2, torch
from google.colab.patches import cv2_imshow
from tqdm.auto import tqdm #tqdm is a library in Python which is used for creating Progress Meters or Progress Bars
import shutil as sh #The shutil module offers a number of high-level operations on files and collection
from IPython.display import Image, clear_output
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from PIL import Image
from base64 import b64encode
from google.colab import files
from moviepy.editor import VideoFileClip
from tensorflow import keras
from keras.models import model_from_json
from flask import Flask, render_template , request 
from werkzeug.utils import secure_filename
from pyngrok import ngrok
model = keras.models.load_model('/content/drive/MyDrive/ds3project/model.h5')

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2940928/45929032 bytes (6.4%)5775360/45929032 bytes (12.6%)10051584/45929032 bytes (21.9%)14295040/45929032 bytes (31.1%)18587648/45929032 bytes (40.5%)22937600/45929032 bytes (49.9%)27320320/45929032 bytes (59.5%)31621120/45929032 bytes (68.8%)35758080/45929032 bytes (77.9%)40165376/45929032 bytes (87.5%)44351488/45929032 bytes (96.6%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

## Data Pre-Processing

In [ ]:
img_h, img_w, num_channels = (380, 676, 3)
df = pd.read_csv('/content/drive/MyDrive/ds3project/data/train_solution_bounding_boxes.csv')
df.rename(columns={'image':'image_id'}, inplace=True)
df['image_id'] = df['image_id'].apply(lambda x: x.split('.')[0])
df['x_center'] = (df['xmin'] + df['xmax'])/2
df['y_center'] = (df['ymin'] + df['ymax'])/2
df['w'] = df['xmax'] - df['xmin']
df['h'] = df['ymax'] - df['ymin']
df['classes'] = 0
df['x_center'] = df['x_center']/img_w
df['w'] = df['w']/img_w
df['y_center'] = df['y_center']/img_h
df['h'] = df['h']/img_h

## Converting Data to YOLO format

In [ ]:
index = list(set(df.image_id)) #creates a set object.
source = 'training_images'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            #print(name,mini)
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('/tmp/convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('/tmp/convertor/fold{}/labels/'.format(fold)+path2save)
            with open('/tmp/convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('/tmp/convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('/tmp/convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("/content/drive/MyDrive/ds3project/data/{}/{}.jpg".format(source,name),'/tmp/convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

  0%|          | 0/355 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Lane Detection Algorithm

In [ ]:
class Lanes():
  def __init__(self):
    self.recent_fit = []
    self.avg_fit = []
lanes = Lanes()
def road_lines(image):
  im = Image.fromarray(image)
  size = tuple((np.array(im.size)).astype(np.uint8))
  small_image  = np.array(im.resize((160,80), Image.BICUBIC))
  small_image = small_image[None,:,:,:]  
  prediction = model.predict(small_image)[0] * 255  # As prediction is between 0 and 1, we multiply by 255 so that it can be used as an image
  lanes.recent_fit.append(prediction)

  if len(lanes.recent_fit) > 5:
    lanes.recent_fit = lanes.recent_fit[1:]
  
  lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis = 0)
  blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)
  lane_drawn = np.dstack((blanks, lanes.avg_fit, blanks))
  lane_image = resize(lane_drawn, image.shape).astype(np.uint8)
  result = cv2.addWeighted(image, 1, lane_image, 1, 0) 
  
  return result

## Object Detection Function

In [ ]:
def detect(file):
  !python yolov5/detect.py --weights yolov5/yolov5s.pt --img 704 --conf 0.4 --source /content/drive/MyDrive/ds3project/test_videos/{file} --exist-ok
  path = 'yolov5/runs/detect/exp/{}'.format(file)
  vidin = VideoFileClip(path)
  lanes = Lanes()
  vid_clip = vidin.fl_image(road_lines)
  vid_clip.write_videofile('/content/drive/MyDrive/ds3project/static/video/{}'.format(file))

## Server Initiation to run the Website

In [ ]:
port_no = 5000
app = Flask(__name__,template_folder='/content/drive/MyDrive/ds3project/templates', static_folder='/content/drive/MyDrive/ds3project/static')
ngrok.set_auth_token("2I6L3d8tO03XZKJUm8aRjcHGyvA_72U4BEyCJgiMT1Y98BKsM")
public_url =  ngrok.connect(port_no).public_url

@app.route("/")  
@app.route('/reg')
def reg():
    return render_template("home.html")

@app.route('/submit', methods = ['POST'])
def submit():
    f = request.files['video']
    detect(secure_filename(f.filename))
    #reading the saved output file and presenting it throught the server
    video_n = "/content/drive/MyDrive/ds3project/static/video/{}".format(f.filename)
    id = !xattr -p 'user.drive.id' {video_n}
    id = "https://drive.google.com/file/d/{}/preview".format(id[0])
    return render_template("view.html", data = input, video = id)

print(f"To acces the Global link please click {public_url}")

app.run(port=port_no)

To acces the Global link please click http://c074-34-68-109-67.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [05/Dec/2022 07:25:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2022 07:25:25] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2022 07:25:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2022 07:25:29] "GET /static/css/home.css HTTP/1.1" 200 -


detect: weights=['yolov5/yolov5s.pt'], source=/content/drive/MyDrive/ds3project/test_videos/stock1.mp4, data=yolov5/data/coco128.yaml, imgsz=[704, 704], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-23-g5dc1ce4 Python-3.8.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

100% 14.1M/14.1M [00:00<00:00, 126MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/393) /content/drive/MyDrive/ds3project/test_videos/stock1.mp4: 416x704 3 persons, 11 cars, 1 truck, 4 traffic lights, 12.9ms
video 1/1 (2/393) /content/drive/MyDrive/ds3project/test_videos/stock1.mp4: 416x704 4 persons, 12 cars, 1 truck, 4 traffic lights, 10.9ms
video

  0%|          | 0/393 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step


...

100%|█████████▉| 392/393 [01:39<00:00,  3.78it/s]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 393/393 [01:39<00:00,  3.96it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/ds3project/static/video/stock1.mp4 



INFO:werkzeug:127.0.0.1 - - [05/Dec/2022 07:28:56] "POST /submit HTTP/1.1" 200 -
